In [1]:
!pip install -q tensorflow-recommenders
!pip install -q scann

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tfx-bsl 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<3,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-serving-api 2.8.0 requires tensorflow<3,>=2.8.0, but you have tensorflow 2.6.3 which is incompatible.
r

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

from pathlib import Path
from typing import Dict, Text

2022-05-07 10:18:27.720311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-05-07 10:18:27.720418: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Preview

In this notebook, we use basic retrieval algorithm. This algorithm is fit for case recommender between two feature. In this competition, we should make recommendation between user and article.

Real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

In this tutorial, we're going to focus on the first stage, retrieval. If you are interested in the ranking stage, have a look at our [ranking](basic_ranking) tutorial.

Retrieval models are often composed of two sub-models:

1. A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
2. A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

In this tutorial, we're going to build and train such a two-tower model using the Movielens dataset.

We're going to:

1. Get our data and split it into a training and test set.
2. Implement a retrieval model.
3. Fit and evaluate it.
4. Export it for efficient serving by building an approximate nearest neighbours (ANN) index.

# Load Data

In [3]:
articles=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customers=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
trans_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', parse_dates=['t_dat'])
submission=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [4]:
articles.head(3)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [5]:
print(f"articles data has {articles.shape[0]} rows and {articles.shape[1]} columns")

articles data has 105542 rows and 25 columns


In [6]:
customers.head(3)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...


In [7]:
print(f"customers data has {customers.shape[0]} rows and {customers.shape[1]} columns")

customers data has 1371980 rows and 7 columns


In [8]:
trans_train.head(3)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2


In [9]:
print(f"transaction data has {trans_train.shape[0]} rows and {trans_train.shape[1]} columns")

transaction data has 31788324 rows and 5 columns


In [10]:
submission.head(3)

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...


From transaction train, we can see, there are article_id and customer_id. These two id is foreign key to customers and articles data. Transaction Table is train data for this model

# Preprocessing

for creating the data, we only use customer_id and article_id in transaction train and article_id in articles data. We slice only customer_id and article_id from transaction train and it becomes customer and we slice articles on article_id and it becomes article

In [11]:
#before slicing, we need convert article_id from int to str, because TensorFlow just allow str
#slicing transaction train into dictionary
trans_train_sample=trans_train[trans_train['t_dat']>='2020-08-01']
trans_train_sample['article_id']=trans_train_sample['article_id'].astype(str)
customer=tf.data.Dataset.from_tensor_slices(dict(trans_train_sample[['customer_id','article_id']]))

#slicing articles into tensor
articles['article_id'] = articles['article_id'].astype(str)
article_ds=tf.data.Dataset.from_tensor_slices(dict(articles[['article_id']]))
article= article_ds.map(lambda x: x['article_id'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
2022-05-07 10:20:22.847209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-05-07 10:20:22.847292: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-07 10:20:22.847323: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be r

To fit and evaluate the model, we need to split it into a training and evaluation set. In an industrial recommender system, this would most likely be done by time: the data up to time $T$ would be used to predict interactions after $T$.


In this simple example, however, let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [12]:
tf.random.set_seed(42)

shuffled = customer.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data. 

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [13]:
#creating unique id
unique_article_id = np.array(articles['article_id'].unique())

unique_customer_id = np.array(customers['customer_id'])

unique_article_id[:10]

array(['108775015', '108775044', '108775051', '110065001', '110065002',
       '110065011', '111565001', '111565003', '111586001', '111593001'],
      dtype=object)

# Implementing a Model

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

## Query Tower

Let's start with the query tower.

The first step is to decide on the dimensionality of the query and candidate representations:

In [14]:
embedding_dimension = 32

Higher values will correspond to models that may be more accurate, but will also be slower to fit and more prone to overfitting.

The second is to define the model itself. Here, we're going to use Keras preprocessing layers to first convert user ids to integers, and then convert those to user embeddings via an `Embedding` layer. Note that we use the list of unique user ids we computed earlier as a vocabulary:

In [15]:
customer_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_customer_id, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_customer_id) + 1, embedding_dimension),
])

A simple model like this corresponds exactly to a classic [matrix factorization](https://ieeexplore.ieee.org/abstract/document/4781121) approach. While defining a subclass of `tf.keras.Model` for this simple model might be overkill, we can easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an `embedding_dimension`-wide output at the end.

## Candidate Tower

We can do the same with the candidate tower.

In [16]:
article_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_article_id, mask_token=None),
  tf.keras.layers.Embedding(len(unique_article_id) + 1, embedding_dimension)
])

## Metrics

In our training data we have positive (customer, article) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.


In [17]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=article.batch(128).map(article_model)
)

## Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

In [18]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.

## Basic Retrieval model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [19]:
class HnMModel(tfrs.Model):

  def __init__(self, customer_model, article_model):
    super().__init__()
    self.article_model: tf.keras.Model = article_model
    self.customer_model: tf.keras.Model = customer_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    customer_embeddings = self.customer_model(features["customer_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    article_embeddings = self.article_model(features["article_id"])

    # The task computes the loss and the metrics.
    return self.task(customer_embeddings, article_embeddings)

The `tfrs.Model` base class is a simply convenience class: it allows us to compute both training and test losses using the same method.

Under the hood, it's still a plain Keras model. You could achieve the same functionality by inheriting from `tf.keras.Model` and overriding the `train_step` and `test_step` functions (see [the guide](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) for details):

In [20]:
class NoBaseClassHnMModel(tf.keras.Model):

  def __init__(self, customer_model, article_model):
    super().__init__()
    self.article_model: tf.keras.Model = article_model
    self.customer_model: tf.keras.Model = customer_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      customer_embeddings = self.customer_model(features["customer_id"])
      artilce_embeddings = self.article_model(features["article_title"])
      loss = self.task(customer_embeddings, article_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    customer_embeddings = self.customer_model(features["custumer_id"])
    article_embeddings = self.artilce_model(features["article_title"])
    loss = self.task(customer_embeddings, article_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

# Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [21]:
model = HnMModel(customer_model, article_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [22]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [23]:
history=model.fit(cached_train, epochs=8)

Epoch 1/8
10/10 [==============================] - 335s 33s/step - factorized_top_k/top_1_categorical_accuracy: 1.7500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0131 - factorized_top_k/top_10_categorical_accuracy: 0.0192 - factorized_top_k/top_50_categorical_accuracy: 0.0389 - factorized_top_k/top_100_categorical_accuracy: 0.0490 - loss: 70350.2543 - regularization_loss: 0.0000e+00 - total_loss: 70350.2543
Epoch 2/8
10/10 [==============================] - 335s 33s/step - factorized_top_k/top_1_categorical_accuracy: 9.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0976 - factorized_top_k/top_10_categorical_accuracy: 0.1409 - factorized_top_k/top_50_categorical_accuracy: 0.2898 - factorized_top_k/top_100_categorical_accuracy: 0.3858 - loss: 69624.3089 - regularization_loss: 0.0000e+00 - total_loss: 69624.3089
Epoch 3/8
10/10 [==============================] - 329s 33s/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_ac

Finally, we can evaluate our model on the test set:

In [24]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 86s 17s/step - factorized_top_k/top_1_categorical_accuracy: 0.0218 - factorized_top_k/top_5_categorical_accuracy: 0.0737 - factorized_top_k/top_10_categorical_accuracy: 0.0869 - factorized_top_k/top_50_categorical_accuracy: 0.1191 - factorized_top_k/top_100_categorical_accuracy: 0.1383 - loss: 33144.6523 - regularization_loss: 0.0000e+00 - total_loss: 33144.6523


{'factorized_top_k/top_1_categorical_accuracy': 0.021800000220537186,
 'factorized_top_k/top_5_categorical_accuracy': 0.07374999672174454,
 'factorized_top_k/top_10_categorical_accuracy': 0.0869000032544136,
 'factorized_top_k/top_50_categorical_accuracy': 0.11905000358819962,
 'factorized_top_k/top_100_categorical_accuracy': 0.13830000162124634,
 'loss': 30133.01171875,
 'regularization_loss': 0,
 'total_loss': 30133.01171875}

In [25]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.customer_model, k = 12 )
scann_index.index_from_dataset(
  tf.data.Dataset.zip((article.batch(100), article.batch(100).map(model.article_model)))
)


_,articles = scann_index(submission.customer_id.values)
preds = articles.numpy().astype(str)
preds = pd.Series(map(' '.join, preds,))

[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"


In [26]:
submission['prediction']=preds
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,877369001 864415005 865880001 817353002 852584...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,505882002 904820002 907696004 698276007 720504...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,832482002 880029001 880029002 747196003 842607...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,745232002 883693001 612800009 479167002 556539...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,918836001 903333001 853472001 881570001 717490...


In [27]:
submission.to_csv('submission.csv', index=False)
print("Submission has been created! CONGRATULATION")

Submission has been created! CONGRATULATION


# References

https://www.kaggle.com/code/viji1609/h-m-basic-retrieval-model-tf-recommender

https://www.kaggle.com/code/beezus666/tensor-flow-recommender-quick-start

https://www.tensorflow.org/recommenders/examples/basic_retrieval